# PermRowCol experimental results

In [15]:
import os

import numpy as np
import pandas as pd
import pyzx as zx

from pyzx import Circuit, cnot_mapper, architecture

First, we specify some code to read the dataset of CNOT circuits that was used in [ArXiv 1904.00633](https://arxiv.org/pdf/1904.00633.pdf).

In [30]:
def read_circuit(source):
    if not os.path.exists(source):
        print("File {} does not exist".format(source))
        return
    return cnot_mapper.CNOT_tracker.from_qasm_file(source)

def read_circuits(n_qubits):
    source_folder = "../circuits/steiner/"
    circuits = []
    sources = []
    subfolder = os.path.join(source_folder, str(n_qubits)+"qubits/")

    for folder in os.listdir(subfolder):
        for file in os.listdir(os.path.join(subfolder, folder)):
            if file.endswith(".qasm"):
                src = os.path.join(subfolder, folder, file)
                circuit = read_circuit(src)
                circuits.append(circuit)
                sources.append(src)
    return circuits, sources

Then we run our experiment with the algorithms as they are.

In [56]:
def run_steiner_gauss(circuit, arch):
    c = cnot_mapper.CNOT_tracker(circuit.n_qubits)
    cnot_mapper.gauss(cnot_mapper.STEINER_MODE, circuit.matrix.copy(), architecture=arch, y=c, full_reduce=True)
    return c.count_cnots()

def run_rowcol(circuit, arch):
    c = cnot_mapper.CNOT_tracker(circuit.n_qubits)
    cnot_mapper.gauss(cnot_mapper.ROWCOL_MODE, circuit.matrix.copy(), architecture=arch, y=c, full_reduce=True)
    return c.count_cnots()

def run_perm_rowcol(circuit, arch):
    c = cnot_mapper.CNOT_tracker(circuit.n_qubits)
    cnot_mapper.permrowcol(circuit.matrix.copy(), architecture=arch, y=c)
    return c.count_cnots()

def run_experiment(arch):
    n_qubits = arch.n_qubits
    og_circuits, srcs = read_circuits(n_qubits)
    results = pd.DataFrame()

    method = {
        "SteinerGauss": run_steiner_gauss,
        "RowCol": run_rowcol,
        "PermRowCol": run_perm_rowcol
    }

    results["Original"] = np.array([int(src.split("/")[-2]) for src in srcs])
    results["#Qubits"] = np.array([n_qubits]*len(og_circuits))
    results["Architecture"] = np.array([arch.name]*len(og_circuits))
    
    for m, func in method.items():
        results[(m, "count")] = np.array([func(c, arch) for c in og_circuits])
        results[(m, "overhead (%)")] = (results[(m, "count")]/results["Original"] - 1)*100

    results = results.groupby(["Original", "Architecture", "#Qubits"]).mean()

    results.columns = pd.MultiIndex.from_tuples(results.columns.tolist())    
    return results
    
archs = [
    architecture.create_architecture(architecture.SQUARE, n_qubits=9),
    architecture.create_architecture(architecture.SQUARE, n_qubits=16),
    architecture.create_architecture(architecture.RIGETTI_16Q_ASPEN),
    architecture.create_architecture(architecture.IBM_QX5),
    architecture.create_architecture(architecture.IBM_Q20_TOKYO)
]

results = pd.concat([run_experiment(arch) for arch in archs])

print(results.head(5))

                              SteinerGauss              RowCol               \
                                     count overhead (%)  count overhead (%)   
Original Architecture #Qubits                                                 
3        9q-square    9              11.50   283.333333  11.45   281.666667   
5        9q-square    9              20.30   306.000000  18.75   275.000000   
10       9q-square    9              32.80   228.000000  30.25   202.500000   
20       9q-square    9              48.25   141.250000  45.25   126.250000   
30       9q-square    9              53.65    78.833333  54.65    82.166667   

                              PermRowCol               
                                   count overhead (%)  
Original Architecture #Qubits                          
3        9q-square    9             9.25   208.333333  
5        9q-square    9            18.05   261.000000  
10       9q-square    9            28.60   186.000000  
20       9q-square    9        

Note that these numbers do not correspond with the original paper for the SteinerGauss algorithm. The reason is that in the paper, we used a genetic algorithm to improve the mapping of the circuit. 
However, we can see that for the algorithms on their own the new PermRowCol algorithm is consistently finding less CNOTs.

Next, we will compare the results with the genetic algorithm for optimizing the initial qubit mapping for the SteinerGauss and RowCol algorithms, and compare that against the PermRowCol combined with a Reverse Traversal strategy for optimizing the mapping.

In [55]:
# TODO